# Importing

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

# Read data

In [3]:
df_ChordFormulae=pd.read_excel("../datasets/allChordsAndFormulae.xlsx")

In [3]:
df_ChordFormulae

,chordName,formula
0,Major,"0,4,7"
1,Minor,"0,3,7"
2,Aug,"0,4,8"
3,Dim,"0,3,6"
4,Fifth,"0,7"
...,...,...
76,Dim7,"0,3,6,9"
77,m#5,"0,3,8"
78,m#7#5,"0,3,8,11"
79,m7#5,"0,3,8,11"


# Analysis

## see duplicate formulae

In [4]:
grpby_df_formula=df_ChordFormulae.groupby('formula')

In [5]:
for index in grpby_df_formula.groups.values():
    if len(index)>1:
        display(df_ChordFormulae.iloc[index])

,chordName,formula
3,Dim,"0,3,6"
64,mb5,"0,3,6"


,chordName,formula
70,tri,"0,3,6,9"
76,Dim7,"0,3,6,9"


,chordName,formula
72,Min#5,"0,3,8"
77,m#5,"0,3,8"


,chordName,formula
78,m#7#5,"0,3,8,11"
79,m7#5,"0,3,8,11"


,chordName,formula
27,7b5,"0,4,6,10"
58,Maj7b5,"0,4,6,10"
74,Dom7b5,"0,4,6,10"


...

,chordName,formula
15,13b9,"0,4,7,10,13"
23,7#5b9,"0,4,7,10,13"
29,7b9,"0,4,7,10,13"


,chordName,formula
12,13,"0,4,7,10,14,21"
34,9b13,"0,4,7,10,14,21"


,chordName,formula
22,7#5#9,"0,4,7,10,15"
24,7#9,"0,4,7,10,15"


,chordName,formula
16,6,"0,4,7,9"
71,Majb7,"0,4,7,9"


,chordName,formula
21,7#5,"0,4,8,10"
75,Dom7#5,"0,4,8,10"


In [7]:
len(grpby_df_formula)

68

### Result

1) there are 68 entries with duplicate formulae

# Convert string formulae to list formulae

In [8]:
def convert_Sf_Lf(lst):
    '''
    info: takes a list of strings and converts it to list of integers
    ip: list of strings
    op: list of integeres
    '''
    rlst=[]
    for i in lst:
        rlst.append(int(i))
    return rlst

In [9]:
df_ChordFormulae['formulaList']=df_ChordFormulae['formula'].str.split(",").apply(convert_Sf_Lf)

In [10]:
df_ChordFormulae.head(3)

,chordName,formula,formulaList
0,Major,"0,4,7","[0, 4, 7]"
1,Minor,"0,3,7","[0, 3, 7]"
2,Aug,"0,4,8","[0, 4, 8]"


# Make chords

In [11]:
baseNotes=["A","A#","B","C","C#","D","D#","E","F","F#","G","G#"]

In [11]:
def makeChord(baseNote,chordName):
    '''
    info: takes a base note and a chord name, returns the notes in the chord
    ip: a base note(string) and a chord name(string)
    op: a list of notes of the chord
    '''
    i=baseNotes.index(baseNote)
    formulaList=df_ChordFormulae.loc[df_ChordFormulae['chordName']==chordName,'formulaList'].values
    chordNotes=[]
    for j in formulaList[0]:
        chordNotes.append(baseNotes[(i+j)%len(baseNotes)])
    return chordNotes

In [24]:
temp=[1,2,6,7,8]
temp.index(7)==len(temp)

False

In [31]:
def makeChord_String(baseNote,chordName):
    '''
    info: takes a base note and a chord name, returns the notes in the chord
    ip: a base note(string) and a chord name(string)
    op: a string of notes of the chord
    '''
    i=baseNotes.index(baseNote)
    formulaList=df_ChordFormulae.loc[df_ChordFormulae['chordName']==chordName,'formulaList'].values
    chordNotes_string=""
    for j in formulaList[0]:
        sep="," if formulaList[0].index(j)!=len(formulaList[0])-1 else ""
        chordNotes_string=chordNotes_string+(baseNotes[(i+j)%len(baseNotes)])+sep
    return chordNotes_string

In [32]:
# make all chords of all base notes
lstIndex_note_chordName=[]
lstAllChordNotes=[]
for note in baseNotes:
    for chordName in df_ChordFormulae['chordName'].unique():
        chordNotes=makeChord_String(note,chordName)
        lstIndex_note_chordName.append((note,chordName))
        lstAllChordNotes.append(chordNotes)

dict1={'indexNoteChord':lstIndex_note_chordName,'ChordNotes':lstAllChordNotes}  
df_allChordNotes=pd.DataFrame(dict1)
display(df_allChordNotes.head(3))

,indexNoteChord,ChordNotes
0,"(A, Major)","A,C#,E"
1,"(A, Minor)","A,C,E"
2,"(A, Aug)","A,C#,F"


In [33]:
df_allChordNotes.to_csv('../datasets/df_chordsData.csv',index=False)

# Analysis

## see duplicate chords

In [17]:
def checkChordEquality(lst1,lst2):
    '''
    info: checks whether list1 is in list2 and vice versa,
          that way even if the lists are shiffled we will be able to compare them
          this is done by converting lists into sets and taking symmetric difference
    ip: 2 lists
    op: if both lists contain same elements then True, o/w False
    '''
    lst1=set(lst1)
    lst2=set(lst2)
    
    if len(lst1.symmetric_difference(lst2))==0:
        return True
    else:
        return False

### test functions

#### checkChordEquality()

In [18]:
lst3=['A', 'C#', 'E']
lst4=['C#', 'E','A','B']
lst5=['A', 'C#', 'E','A']
lst6=['A', 'C#','A']
lst7=['C#','A','E']

In [19]:
checkChordEquality(lst3,lst5)

True

In [20]:
checkChordEquality(lst3,lst4)

False

In [21]:
checkChordEquality(lst3,lst6)

False

In [22]:
checkChordEquality(lst3,lst7)

True

### make groups of similar chords

In [23]:
df_allChordNotes.head()

,indexNoteChord,ChordNotes
0,"(A, Major)","[A, C#, E]"
1,"(A, Minor)","[A, C, E]"
2,"(A, Aug)","[A, C#, F]"
3,"(A, Dim)","[A, C, D#]"
4,"(A, Fifth)","[A, E]"


In [24]:
i=1
df_allChordNotes.loc[i,'ChordNotes']

['A', 'C', 'E']

In [25]:
dict_similarChords={}
doneIndexes=[]
for i in range(len(df_allChordNotes.index)):
    if i not in doneIndexes:
        groupElements=[]
        for j in range(i,len(df_allChordNotes.index)):
            lst1=df_allChordNotes.loc[i,'ChordNotes']
            lst2=df_allChordNotes.loc[j,'ChordNotes']
            if checkChordEquality(lst1,lst2):
                doneIndexes.append(j)
                groupElements.append(j)
        dict_similarChords.update({df_allChordNotes.loc[i,'indexNoteChord']:groupElements})

In [26]:
len(dict_similarChords)

577

In [27]:
dict_similarChords

{('A', 'Major'): [0, 396, 401],
 ('A', 'Minor'): [1, 605],
 ('A', 'Aug'): [2, 326, 650],
 ('A', 'Dim'): [3, 64],
 ('A', 'Fifth'): [4, 6],
 ('A', 'Fifth 9th'): [5, 8, 576],
 ('A', 'Octave'): [7],
 ('A', 'Sus4'): [9, 410, 413],
 ('A', 11): [10],
 ('A', '11b9'): [11],
 ('A', 13): [12, 34],
 ('A', '13#9'): [13],
 ('A', '13b5b9'): [14],
 ('A', '13b9'): [15, 23, 29],
 ('A', 6): [16, 71, 772],
 ('A', '6add9'): [17, 198, 773],
 ('A', '6sus4'): [18, 442],
 ('A', 7): [19, 73],
 ('A', '7#11'): [20, 514],
 ('A', '7#5'): [21, 75],
 ('A', '7#5#9'): [22, 24],
 ('A', '7add11'): [25, 467],
 ('A', '7add13'): [26, 776],
 ('A', '7b5'): [27, 58, 74, 513, 544, 560],
 ('A', '7b5b9'): [28, 506],
 ('A', '7sus4'): [30],
 ('A', 9): [31],
 ('A', '9#11'): [32],
 ('A', '9#5'): [33, 845],
 ('A', '9b5'): [35, 195],
 ('A', '9sus4'): [36, 611, 827],
 ('A', 'add9'): [37, 585],
 ('A', 'augsus4'): [38, 406],
 ('A', 'm11'): [39, 296],
 ('A', 'm13'): [40],
 ('A', 'm6'): [41, 775],
 ('A', 'm6add9'): [42],
 ('A', 'm7'): [43, 

In [28]:
df_allChordNotes.head()

,indexNoteChord,ChordNotes
0,"(A, Major)","[A, C#, E]"
1,"(A, Minor)","[A, C, E]"
2,"(A, Aug)","[A, C#, F]"
3,"(A, Dim)","[A, C, D#]"
4,"(A, Fifth)","[A, E]"


In [29]:
for index in dict_similarChords.values():
    if len(index)>1:
        display(df_allChordNotes.iloc[index])

,indexNoteChord,ChordNotes
0,"(A, Major)","[A, C#, E]"
396,"(C#, Min#5)","[C#, E, A]"
401,"(C#, m#5)","[C#, E, A]"


,indexNoteChord,ChordNotes
1,"(A, Minor)","[A, C, E]"
605,"(E, augsus4)","[E, A, C]"


,indexNoteChord,ChordNotes
2,"(A, Aug)","[A, C#, F]"
326,"(C#, Aug)","[C#, F, A]"
650,"(F, Aug)","[F, A, C#]"


,indexNoteChord,ChordNotes
3,"(A, Dim)","[A, C, D#]"
64,"(A, mb5)","[A, C, D#]"


,indexNoteChord,ChordNotes
4,"(A, Fifth)","[A, E]"
6,"(A, Fifth Octave)","[A, E, A]"


...

,indexNoteChord,ChordNotes
906,"(G#, 13b9)","[G#, C, D#, F#, A]"
914,"(G#, 7#5b9)","[G#, C, D#, F#, A]"
920,"(G#, 7b9)","[G#, C, D#, F#, A]"


,indexNoteChord,ChordNotes
910,"(G#, 7)","[G#, C, D#, F#]"
964,"(G#, Dom7)","[G#, C, D#, F#]"


,indexNoteChord,ChordNotes
912,"(G#, 7#5)","[G#, C, E, F#]"
966,"(G#, Dom7#5)","[G#, C, E, F#]"


,indexNoteChord,ChordNotes
913,"(G#, 7#5#9)","[G#, C, D#, F#, B]"
915,"(G#, 7#9)","[G#, C, D#, F#, B]"


,indexNoteChord,ChordNotes
969,"(G#, m#7#5)","[G#, B, E, G]"
970,"(G#, m7#5)","[G#, B, E, G]"


In [30]:
len(dict_similarChords)

577

# Create Scale data

degrees in a scale i.e. notes in a scale are

1st degree = tonic

2nd degree = supertonic

3rd degree = mediant

4th degree = subdominant

5th degree = dominant

6th degree = submediant

7th degree = leading tone

8th degree = octave high of tonic

In [31]:
scale_formula_major=[0,2,4,5,7,9,11,12]
scale_formula_naturalMinor=[0,2,3,5,7,8,10,12]
scale_formula_harmonicMinor=[0,2,3,5,7,8,11,12]
scale_formula_melodicMinor=[0,2,3,5,7,9,11,12]
scale_formula_dorianb2=[0,1,3,5,7,9,11,12]
scale_formula_phrygingDominant=[0,1,4,5,7,8,11,12]
scale_formula_lydianMinor=[0,2,4,6,7,8,10,12]
scale_formula_mixolydianAugmented=[0,2,4,5,8,9,10,12]
scale_formula_locrianDominant=[0,2,4,5,6,8,10,12]

In [32]:
# create this dictinorary from a database later
dict_scaleName_formula={'Major':scale_formula_major,'Natural Minor':scale_formula_naturalMinor,\
                        'Harmonic Minor':scale_formula_harmonicMinor,\
                        'Melodic Minor':scale_formula_melodicMinor,'Dorianb2':scale_formula_dorianb2,\
                        'Phyryging Dominant':scale_formula_phrygingDominant,\
                        'Lydian Minor':scale_formula_lydianMinor,'Mixolydian Augmented':scale_formula_mixolydianAugmented,\
                        'Locrian Dominant':scale_formula_locrianDominant}

In [33]:
baseNotes

['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']

In [34]:
class Scale():
    def __init__(self, name, baseNote):
        self.name=name
        self.baseNote=baseNote
        self.notes = self.populateNotes()
        
    def populateNotes(self):
        temp=[]
        for i in dict_scaleName_formula[self.name]:
            #print(i)
            index_baseNote=baseNotes.index(self.baseNote)
            temp.append(baseNotes[(index_baseNote+i)%len(baseNotes)])
        #print(temp)
        return temp

In [35]:
scale=Scale('Major','C')

In [36]:
temp=[1,2,3]

In [37]:
str(temp)

'[1, 2, 3]'

In [38]:
temp="[1,2,3]"

In [39]:
temp

'[1,2,3]'

In [40]:
temp.rstrip("]")

'[1,2,3'

In [41]:
temp.lstrip("[]")

'1,2,3]'

In [42]:
temp.strip("[").strip("]").split(",")

['1', '2', '3']

In [43]:
temp

'[1,2,3]'

In [44]:
def updateDb_scales():
    dict_scaleName_formula_db={'Note':[],'Scale':[],'Notes':[]}
    for baseNote in baseNotes:
        for name in dict_scaleName_formula.keys():
            dict_scaleName_formula_db['Note'].append(baseNote)
            dict_scaleName_formula_db['Scale'].append(name)
            dict_scaleName_formula_db['Notes'].append(str(Scale(name, baseNote).populateNotes()).strip("[").strip("]"))
    return dict_scaleName_formula_db

In [45]:
A = updateDb_scales()
import pandas as pd
df_scalesData=pd.DataFrame(A)

In [46]:
df_scalesData.head()

,Note,Scale,Notes
0,A,Major,"'A', 'B', 'C#', 'D', 'E', 'F#', 'G#', 'A'"
1,A,Natural Minor,"'A', 'B', 'C', 'D', 'E', 'F', 'G', 'A'"
2,A,Harmonic Minor,"'A', 'B', 'C', 'D', 'E', 'F', 'G#', 'A'"
3,A,Melodic Minor,"'A', 'B', 'C', 'D', 'E', 'F#', 'G#', 'A'"
4,A,Dorianb2,"'A', 'A#', 'C', 'D', 'E', 'F#', 'G#', 'A'"


In [52]:
df_scalesData.to_csv('../datasets/df_scalesData.csv',index=False)

In [53]:
import pandas as pd
df_scaledData=pd.read_csv('../datasets/df_scalesData.csv')

In [54]:
df_scaledData.dtypes

Note     object
Scale    object
Notes    object
dtype: object

In [55]:
df_scaledData

,Note,Scale,Notes
0,A,Major,"'A', 'B', 'C#', 'D', 'E', 'F#', 'G#', 'A'"
1,A,Natural Minor,"'A', 'B', 'C', 'D', 'E', 'F', 'G', 'A'"
2,A,Harmonic Minor,"'A', 'B', 'C', 'D', 'E', 'F', 'G#', 'A'"
3,A,Melodic Minor,"'A', 'B', 'C', 'D', 'E', 'F#', 'G#', 'A'"
4,A,Dorianb2,"'A', 'A#', 'C', 'D', 'E', 'F#', 'G#', 'A'"
...,...,...,...
103,G#,Dorianb2,"'G#', 'A', 'B', 'C#', 'D#', 'F', 'G', 'G#'"
104,G#,Phyryging Dominant,"'G#', 'A', 'C', 'C#', 'D#', 'E', 'G', 'G#'"
105,G#,Lydian Minor,"'G#', 'A#', 'C', 'D', 'D#', 'E', 'F#', 'G#'"
106,G#,Mixolydian Augmented,"'G#', 'A#', 'C', 'C#', 'E', 'F', 'F#', 'G#'"


In [51]:
df_scaledData['Notes'][0].strip("[").strip("]").split(", ")

["'A'", "'B'", "'C#'", "'D'", "'E'", "'F#'", "'G#'", "'A'"]

In [ ]:
# get simiilar scales

# Plan

1) create scales data

2) read it

3) make connection between chords and scales

4) maybe generate a data which contains
 * scale name
 * scale formula
 * all chords in it
 
5) find note similar scales

6) you have around 577 similar chords, unique. these are dimensions, or kinda "words", scqales are "sentences". Or some analogy like this.

7) 2 scales are chord similar if both of them contain same chords, ypu can have percentage similarity here

8) do this percentage thing for note similarity too.
 * make groups like differ by 1 note, differ by 2 notes, etc...
 
9) see if these 2 similarities are correlated

10) suggest chords based on these similarities

11) plans for mood:
    
 1. each scale has mood
 2. but mood depends on how you travel via chords
 3. mood depends on what chords you have recently played.
  * you cant say C major has positive vibes, because if you are in E major scale and suddenly you play C major, its gonna change mood